In [1]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

# Basic autograd example 1

In [8]:
# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

# compute gradients
y.backward()

# print out the gradients
print(x.grad)
print(w.grad)
print(b.grad)

tensor(2.)
tensor(1.)
tensor(1.)


# Basic autograd example 2

In [9]:
# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# Build a fully connected layer.
# it applies a linear transformation to the incoming data: y = x.W^T +b
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

w:  Parameter containing:
tensor([[-0.1226, -0.4654,  0.0720],
        [ 0.2753,  0.2620, -0.0379]], requires_grad=True)
b:  Parameter containing:
tensor([-0.4172,  0.0532], requires_grad=True)


In [10]:
# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

In [11]:
# Forward pass.
pred = linear(x)
pred

tensor([[-0.6236,  0.0779],
        [-0.7050,  0.5062],
        [-0.5580,  0.1945],
        [-0.3094,  0.0172],
        [-0.4416,  0.1251],
        [-1.2312,  0.4324],
        [-0.7521,  0.5728],
        [-0.3195,  0.0876],
        [-0.7464,  0.4183],
        [-0.2564,  0.2652]], grad_fn=<AddmmBackward0>)

In [12]:
# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.item())

loss:  1.4800628423690796


In [13]:
# Backward pass.
loss.backward()

In [14]:
# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

dL/dw:  tensor([[ 0.0250, -0.4512,  0.0589],
        [ 0.2959, -0.0659,  0.1853]])
dL/db:  tensor([-0.0948,  0.1248])


In [18]:
# 1-step gradient descent.
optimizer.step()
print ('w: ', linear.weight)
print ('b: ', linear.bias)

w:  Parameter containing:
tensor([[-0.1234, -0.4519,  0.0702],
        [ 0.2665,  0.2640, -0.0435]], requires_grad=True)
b:  Parameter containing:
tensor([-0.4144,  0.0494], requires_grad=True)


In [ ]:
# You can also perform gradient descent at the low level.
linear.weight.data.sub_(0.01 * linear.weight.grad.data)
linear.bias.data.sub_(0.01 * linear.bias.grad.data)

In [19]:
# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

loss after 1 step optimization:  1.469442367553711


# Loading data from numpy 

In [20]:
# Create a numpy array.
x = np.array([[1, 2], [3, 4]])

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

# Convert the torch tensor to a numpy array.
z = y.numpy()

# Input pipeline  

In [22]:
# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.MNIST(root='./data/',
                                            train=True, 
                                            transform=transforms.ToTensor(),
                                            download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [23]:
# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

torch.Size([1, 28, 28])
5


In [24]:
# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

In [25]:
# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

In [ ]:
# Mini-batch images and labels.
images, labels = data_iter.next()
print(images.shape)

In [27]:
# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass

# Input pipeline for custom dataset   

In [29]:
# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 

In [ ]:
# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)

# Pretrained model

In [ ]:
# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)

# Save and load the model

In [ ]:
# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))